In [4]:
# Check all downloaded models with transformers module

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill").to(device)
model.eval()

KeyboardInterrupt: 

In [7]:
import pandas as pd

def sample_df():
    """Create a small sample DataFrame for testing"""
    # Load the data from /home/zahemen/projects/dl-lib/DocAnalyzerAI/data/AAPL/AAPL_financials.csv and set 'Unnamed: 0' as the index
    return pd.read_csv('/home/zahemen/projects/dl-lib/DocAnalyzerAI/data/AAPL/AAPL_financials.csv', index_col='Unnamed: 0')

df = sample_df()
df.head()

,Net sales,Cost of sales,Gross margin,Research and development,"Selling, general and administrative",Total operating expenses,Operating income,Income before provision for income taxes,Provision for income taxes,Net income,...,"Payments for acquisition of property, plant and equipment",Cash generated by/(used in) investing activities,Payments for dividends and dividend equivalents,Repurchases of common stock,"Proceeds from issuance of term debt, net",Repayments of term debt,"Proceeds from/(Repayments of) commercial paper, net",Cash used in financing activities,"Increase/(Decrease) in cash, cash equivalents, and restricted cash and cash equivalents","Cash paid for income taxes, net"
2024,391035000000,-210352000000,180683000000,31370000000,26097000000,-57467000000,123216000000,123485000000,-29749000000,93736000000,...,-9447000000,2935000000,-15234000000,-94949000000,0,-9958000000,3960000000,-121983000000,-794000000,26102000000
2023,383285000000,-214137000000,169148000000,29915000000,24932000000,-54847000000,114301000000,113736000000,-16741000000,96995000000,...,-10959000000,3705000000,-15025000000,-77550000000,5228000000,-11151000000,-3978000000,-108488000000,5760000000,18679000000
2022,394328000000,-223546000000,170782000000,26251000000,25094000000,-51345000000,119437000000,119103000000,-19300000000,99803000000,...,-10708000000,-22354000000,-14841000000,-89402000000,5465000000,-9543000000,3955000000,-110749000000,-10952000000,19573000000


In [5]:
import sys
sys.path.append('../src')

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from table_analyzer import TableAnalyzer
import re
import logging
from typing import Dict, List, Tuple, Optional

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FinancialChatbot:
    def __init__(self, model_name="facebook/blenderbot-400M-distill"):
        """Initialize chatbot with financial analysis capabilities"""
        # Initialize base model
        self._init_model(model_name)
        
        # Initialize table analyzer
        self.analyzer = TableAnalyzer()
        
        # State management
        self.conversation_history = []
        self.current_context = {}
        self.last_metric_discussed = None
        
        # Define financial query patterns
        self.query_patterns = {
            'growth_rate': r'growth.*(rate|percentage).*(\d{4})',
            'metric_value': r'(what|how much).*(revenue|sales|income|profit).*(\d{4})',
            'comparison': r'compare.*(revenue|sales|income|profit).*(\d{4}).*(\d{4})',
            'trend': r'trend.*(revenue|sales|income|profit)',
        }
        
        # Financial metric explanations
        self.metric_explanations = {
            'net_sales': "Net sales represents the company's total revenue from goods and services",
            'operating_income': "Operating income shows profit from core business operations",
            'net_income': "Net income is the company's total profit after all expenses and taxes",
            'research_development': "R&D expenses show investment in future innovation",
        }

    def _init_model(self, model_name: str) -> None:
        """Initialize the base conversation model"""
        try:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            self.model.eval()
        except Exception as e:
            logger.error(f"Model initialization error: {e}")
            raise

    def _identify_query_type(self, query: str) -> str:
        """Identify the type of financial query"""
        query = query.lower()
        for query_type, pattern in self.query_patterns.items():
            if re.search(pattern, query):
                return query_type
        return 'general'

    def _extract_metrics_and_years(self, query: str) -> Tuple[List[str], List[str]]:
        """Extract financial metrics and years from query"""
        metrics = re.findall(r'(revenue|sales|income|profit|research|development)', query.lower())
        years = re.findall(r'\b(20\d{2})\b', query)
        return metrics, years

    def _format_financial_response(self, value: float, metric: str) -> str:
        """Format financial values with explanations"""
        formatted_value = f"${value:,.2f}"
        explanation = self.metric_explanations.get(metric, "")
        return f"{formatted_value}. {explanation}"

    def _generate_trend_analysis(self, metric: str, data: Dict) -> str:
        """Generate trend analysis for a metric"""
        values = [float(v.strip('%')) for v in data.values()]
        trend = "increasing" if sum(values) > 0 else "decreasing"
        avg_change = sum(values) / len(values)
        return f"The {metric} shows a {trend} trend with an average change of {avg_change:.2f}% per year."

    def process_financial_query(self, query: str, df) -> str:
        """Process financial queries using TableAnalyzer"""
        try:
            query_type = self._identify_query_type(query)
            metrics, years = self._extract_metrics_and_years(query)
            
            if query_type == 'growth_rate' and metrics:
                metric = metrics[0]
                growth_rates = self.analyzer.calculate_growth_rate(df, metric)
                self.last_metric_discussed = metric
                return f"Growth rates for {metric}: {growth_rates}"
                
            elif query_type == 'metric_value':
                processed_query = self.analyzer.preprocess_query(query)
                answer = self.analyzer.query_table(df, processed_query)
                return answer
                
            elif query_type == 'trend':
                metric = metrics[0] if metrics else self.last_metric_discussed
                if metric:
                    growth_rates = self.analyzer.calculate_growth_rate(df, metric)
                    return self._generate_trend_analysis(metric, growth_rates)
                    
            return self.analyzer.query_table(df, query)
            
        except Exception as e:
            logger.error(f"Error processing financial query: {e}")
            return "I apologize, but I couldn't process that financial query. Could you rephrase it?"

    def chat(self, user_input: str, financial_data=None) -> str:
        """Main chat interface with context management"""
        try:
            # Store conversation context
            self.conversation_history.append(("user", user_input))
            
            # Check if it's a financial query
            if financial_data is not None:
                response = self.process_financial_query(user_input, financial_data)
                if response and "Error" not in response:
                    self.conversation_history.append(("bot", response))
                    return response
            
            # Fall back to general conversation
            inputs = self.tokenizer(user_input, return_tensors="pt").to(self.device)
            with torch.no_grad():
                outputs = self.model.generate(**inputs, max_length=100)
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            self.conversation_history.append(("bot", response))
            return response
            
        except Exception as e:
            logger.error(f"Chat error: {e}")
            return "I apologize, but I encountered an error. Could you try asking another way?"

    def suggest_related_queries(self, last_metric: Optional[str] = None) -> List[str]:
        """Suggest related queries based on conversation context"""
        suggestions = []
        if last_metric:
            suggestions.extend([
                f"What's the growth rate for {last_metric}?",
                f"Can you show the trend in {last_metric}?",
                f"Compare {last_metric} between different years"
            ])
        return suggestions

    def reset_conversation(self) -> None:
        """Reset conversation state"""
        self.conversation_history = []
        self.current_context = {}
        self.last_metric_discussed = None

def main():
    print("Initializing chatbot... (this may take a moment)")
    chatbot = FinancialChatbot()
    print("Chatbot is ready! Type 'quit' to exit or 'reset' to start a new conversation.")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'reset':
            chatbot.reset_conversation()
            print("Chat history has been reset.")
            continue
        
        if user_input:
            response = chatbot.chat(user_input)
            print(f"Bot: {response}")

if __name__ == "__main__":
    main()


Initializing chatbot... (this may take a moment)


INFO:table_analyzer:Initializing TAPAS model...
INFO:table_analyzer:TAPAS model ready!


Chatbot is ready! Type 'quit' to exit or 'reset' to start a new conversation.
Bot:  Hey, how are you? I'm just hanging out with my dog, he's my best friend.
Bot:  Do you know much about analyzing financial data? It's a very interesting topic.
Bot:  Have you ever been on an airplane? I love to exit from one place to another.


In [9]:
from pathlib import Path
import pandas as pd

def test_financial_chat():
    # Initialize chatbot
    chatbot = FinancialChatbot()
    
    # Load financial data
    data_path = Path("../data/AAPL/AAPL_financials.csv")
    financial_data = pd.read_csv(data_path)
    
    # Test queries
    test_queries = [
        "What was the revenue in 2024?",
        "Show me the growth rate for net income",
        "What's the trend in operating income?",
        "Compare research and development expenses between 2023 and 2024",
        "Can you explain what net income means?"
    ]
    
    print("\nTesting Financial Chatbot:")
    print("-" * 50)
    
    for query in test_queries:
        print(f"\nUser: {query}")
        response = chatbot.chat(query, financial_data)
        print(f"Bot: {response}")
        
        # Show suggested follow-up queries
        suggestions = chatbot.suggest_related_queries(chatbot.last_metric_discussed)
        if suggestions:
            print("\nSuggested queries:")
            for suggestion in suggestions:
                print(f"- {suggestion}")

if __name__ == "__main__":
    test_financial_chat()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:table_analyzer:Initializing TAPAS model...
INFO:table_analyzer:TAPAS model ready!
/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys a


Testing Financial Chatbot:
--------------------------------------------------

User: What was the revenue in 2024?
Bot: 93,736,000,000

User: Show me the growth rate for net income


/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]
ERROR:table_analyzer:Error calculating growth rate: 'income'
ERROR:__main__:Error processing financial query: division by zero


Bot: Could not find an answer in the table

User: What's the trend in operating income?
Bot: I apologize, but I couldn't process that financial query. Could you rephrase it?

User: Compare research and development expenses between 2023 and 2024


/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Bot: 2

User: Can you explain what net income means?


/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/home/zahemen/miniconda3/envs/transformer_env/lib/python3.11/site-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


Bot: 93,736,000,000
